# Custom sliders

In [ ]:
%matplotlib notebook
from pyfitit import *
initPyfitit()

## Read settings and sample

In [ ]:
expSpectrum = readSpectrum('../Feterpy inverse method/exp_excited.txt')
sample = readSample('sample_result')

## Define function to be optimize by sliders

The function not only calculate norm between theory and experiment for given parameters, but also add data to slf.data for plotting and saving

In [ ]:
def xanesPredictorFunc(slf, params):
    energyRange = params['energyRange']
    slf.data['exp'] = FuncModel.createDataItem('plot', x=expSpectrum.energy, y=expSpectrum.intensity, order=-1, color='black', lw=2)
    methodName = params['method']

    def getFittedEstimator():
        method = inverseMethod.getMethod(methodName)
        estimator = ML.Normalize(method, xOnly=False)
        estimator.fit(sample.params.to_numpy(), sample.spectra.to_numpy())
        return estimator
    estimator = slf.cache.getFromCacheOrEval(dataName='estimator', evalFunc=getFittedEstimator, dependData=[methodName])
    geomParams = [params[pn] for pn in sample.paramNames]
    sp = slf.cache.getFromCacheOrEval(dataName='prediction', evalFunc=lambda: utils.Spectrum(sample.energy, estimator.predict(np.array(geomParams).reshape(1, -1)).reshape(-1)), dependData=[methodName, *geomParams])
    shift = params['shift']
    smoothed, norm = smoothLib.smoothInterpNorm(smoothParams=params, spectrum=sp, smoothType='fdmnes', expSpectrum=expSpectrum, fitNormInterval=energyRange)
    slf.data['not smoothed'] = FuncModel.createDataItem('plot', x=sp.energy + shift, y=sp.intensity / norm)
    slf.data['theory'] = FuncModel.createDataItem('plot', x=smoothed.energy, y=smoothed.intensity, order=1)

    def moleculeConstructor(params):
        mol = Molecule('../Feterpy inverse method/Fe_terpy.xyz')
        mol.atom[1,1] -= params['shift1']
        mol.atom[30,1] += params['shift2']
        return mol

    # add molecule, which is lazy generated (only when we save data)
    slf.data['molecule'] = FuncModel.createDataItem('lazy', 
        generator=lambda: FuncModel.createDataItem('text', str=moleculeConstructor(params).export_xyz_string(), 
                                                   filePostfix='[label].xyz', plot=False))

    def xlim(ax):
        ax.set_xlim(energyRange)
        plotting.updateYLim(ax)
    slf.data['xlim'] = FuncModel.createDataItem('custom', plotter=xlim, save=False, order=1000)
    error = utils.rFactorSp(smoothed, expSpectrum, p=1, sub1=True, interval=energyRange)
    slf.data['error'] = FuncModel.createDataItem('text', str='err = %.3g' % error, order=1001)
    return error

## Define paramProperties class with param bounds (domains) and default values

In [ ]:
paramProperties = ParamProperties()
paramProperties['shift1'] = {'type':'float', 'domain':[-0.1, 0.1]}
paramProperties['shift2'] = {'type':'float', 'domain':[-0.1, 0.1]}
paramProperties['Gamma_hole'] = {'type':'float', 'domain':[0.1, 10], 'default':4.23}
paramProperties['Ecent'] = {'type':'float', 'domain':[5, 100], 'default':53}
paramProperties['Elarg'] = {'type':'float', 'domain':[5, 100], 'default':24}
paramProperties['Gamma_max'] = {'type':'float', 'domain':[10, 30], 'default':24}
paramProperties['Efermi'] = {'type':'float', 'domain':[7000, 7120], 'default':7115}
paramProperties['shift'] = {'type':'float', 'domain':[125, 145], 'default':136}
paramProperties['norm'] = {'type':'float', 'domain':[0, 0.05], 'default':0.0323}
paramProperties['energyRange'] = {'type':'range', 'domain':[expSpectrum.energy[0], expSpectrum.energy[-1]]}
paramProperties['method'] = {'type': 'list', 'domain': inverseMethod.allowedMethods, 'default':'RBF'}

## Construnct FuncModel class and sliders widget

In [ ]:
funcModel = FuncModel(name='Feterpy XANES fit', function=xanesPredictorFunc, paramProperties=paramProperties)
result = FuncModelSliders(funcModel=funcModel, fitType='min', defaultParams={'fit Gamma_hole':False, 'fit Ecent':False, 'fit Elarg':False, 
                        'fit Gamma_max':False, 'fit Efermi':False, 'fit shift':False, 'fit norm':False})

In [ ]:
result.saveAllData('results/fit_result.txt')